# Laboratorio de regresión - 5   Ej 1   Arturo Ayala Hernández

|                |                        |
:----------------|------------------------|
| **Nombre**     | Arturo Ayala Hernández |
| **Fecha**      | 16/02/26               |
| **Expediente** | 758742                 |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Porque necesitamos probar el modelo con datos que NO haya visto antes.

Si el modelo estudia con las respuestas a la mano, sacará 100 en el examen pero no habrá aprendido nada (eso se llama overfitting). Al separar los datos, simulamos un examen real para ver si de verdad aprendió a predecir.

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

Para aprender sí, pero para evaluar NO.

Usar todos los datos es arriesgado porque te quedas sin forma de comprobar si tu modelo funciona.

La solución ideal: Usamos validación cruzada (como en este ejercicio) para usar casi todos los datos y aun así poder evaluarlo. Ya que estamos seguros de que funciona, al final sí podemos entrenarlo con todo para usarlo en la vida real.

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

### 1. Carga de librerías y preprocesamiento
Cargamos el dataset `Motor Trend Car Road Tests.xlsx` y eliminamos la columna `model` (texto) para quedarnos solo con las variables numéricas.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Carga de datos
df = pd.read_excel('Motor Trend Car Road Tests.xlsx')

# Eliminar la columna 'model'
df_clean = df.drop(columns=['model'])

# Mostrar las primeras filas para verificar
print(df_clean.head())

    mpg  cyl   disp   hp  drat     wt   qsec  vs  am  gear  carb
0  21.0    6  160.0  110  3.90  2.620  16.46   0   1     4     4
1  21.0    6  160.0  110  3.90  2.875  17.02   0   1     4     4
2  22.8    4  108.0   93  3.85  2.320  18.61   1   1     4     1
3  21.4    6  258.0  110  3.08  3.215  19.44   1   0     3     1
4  18.7    8  360.0  175  3.15  3.440  17.02   0   0     3     2


### 2. Definición de variables
Separamos la variable objetivo $y$ (`mpg`) de las variables predictoras $X$ (todo lo demás). Convertimos los datos a arreglos de `numpy` para facilitar la manipulación manual de los índices.

In [2]:
# Variables predictoras (X) y Target (y)
X = np.array(df_clean.drop(columns=['mpg']))
y = np.array(df_clean['mpg'])

# número total de muestras (n)
n = len(X)
print(f"Total de muestras (autos): {n}")

Total de muestras (autos): 32


### 3.  Leave-One-Out CV
Realizamos un ciclo que se repite $n$ veces (32 iteraciones). En cada vuelta:
1.  **Separamos** un auto para prueba (Test) y usamos los 31 restantes para entrenar (Train).
2.  **Entrenamos** el modelo de Regresión Lineal.
3.  **Predecimos** el valor del auto oculto.
4.  **Calculamos** el Error Cuadrático (MSE) y lo guardamos.

In [3]:
# Lista para guardar los errores
errores_individuales = []

print("Iniciando validación cruzada...")

for i in range(n):
    # --- Paso A: Separar Train y Test ---
    # Test: El auto 'i' (usamos reshape porque es un solo dato)
    X_test = X[i].reshape(1, -1)
    y_test = y[i].reshape(1, -1)

    # Train: Todos los autos MENOS el 'i' (axis=0 borra la fila)
    X_train = np.delete(X, i, axis=0)
    y_train = np.delete(y, i, axis=0)

    # --- Paso B: Entrenar Modelo ---
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    # --- Paso C: Predecir ---
    prediccion = lr.predict(X_test)

    # --- Paso D: Calcular Error ---
    error = mean_squared_error(y_test, prediccion)
    errores_individuales.append(error)

print("Validación completada.")

Iniciando validación cruzada...
Validación completada.


### 4. Resultados Generales
Calculamos el promedio de los errores obtenidos en cada iteración para obtener el **MSE Global** del modelo.

In [4]:
# Promedio de todos los errores
mse_promedio = np.mean(errores_individuales)

print(f"MSE Promedio (Mean Squared Error): {mse_promedio}")
print(f"Raíz del MSE (RMSE): {np.sqrt(mse_promedio)}")

MSE Promedio (Mean Squared Error): 12.181558006901948
Raíz del MSE (RMSE): 3.490208877259633


# Interpretación

Interpretación:

El resultado nos dio un RMSE de 3.49.

Esto significa que, cada vez que intentamos predecir el rendimiento de un auto nuevo, el modelo se equivoca (en promedio) por unas 3.5 millas por galón. Es el margen de error normal de este modelo.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [5]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']


Interpreta.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3